# Deploy Container Image
| ECR에 푸시된 커스텀 컨테이너 이미지를 이용해서 Sagemaker Endpoint에 등록하는 과정을 안내합니다.


## 개요
커스텀 컨테이너 이미지를 Sagemaker Endpoint로 등록하는 과정은 세 단계로 이루어집니다.

1. Sagemaker Model 등록하기
2. Sagemaker Endpoint Config 등록하기
3. Sagemaker Endpoint 등록하기

In [ ]:
import boto3
client = boto3.client('sagemaker')

In [ ]:
# Create Sagemaker Model
create_model_response = client.create_model(
    ModelName='<생성할 Sagemaker 모델명>', # ex) sm-dummy-inference-model
    PrimaryContainer={ #
        'Image': '<ECR 레포지토리 URI>', # ex) 9803771.dkr.ecr.ap-northeast-2.amazonaws.com/dummy-inference-model',
        'Environment': { }
    },
    ExecutionRoleArn='<AmazonSagemakerFullAccess를 포함하는 IAM 역할>' # ex) arn:aws:iam::980377:role/service-role/AmazonSageMaker-ExecutionRole-20231127T170565'
)

print("create model response:", create_model_response)

In [ ]:
# Create Sagemaker Endpoint Config
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName='<엔드포인트 컨피그명>', # ex) dummy-inference-serve-config',
    ProductionVariants=[
        {
            'ModelName': '<앞에서 정의한 Sagemaker 모델명>', # ex) sm-dummy-inference-model
            'VariantName': '<변형 명>', # ex) variant-sm-dummy-inference-model-1
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        }
    ]
)

print('create endpoint config response:', create_endpoint_config_response)

In [ ]:
# Create Sagemaker Endpoint
create_endpoint_response = client.create_endpoint(
    EndpointName='<엔드포인트 명>', # ex) dummy-inference-serve',
    EndpointConfigName='<직전에 정의한 엔드포인트 컨피그명>', # ex) dummy-inference-serve-config'
)

print('create endpoint response: ', create_endpoint_response)

In [ ]:
# Sagemaker Endpoint 호출하기
sagemaker_runtime = boto3.client("runtime.sagemaker")
inference_response = sagemaker_runtime.invoke_endpoint(
    EndpointName='<엔드포인트 명>',
    Body="Hello World"
)

response_str = inference_response["Body"].read().decode()
print(response_str)